# About 

This is just a practice sandbox to understand the use of Mock and pytest

## References

[Understanding Unittest.Mock
 by Mario Corchero[Video]](https://learning.oreilly.com/videos/understanding-unittest-mock/9781484244135)
 
 [Hands-On Test Driven Development with Python [Video]](https://www.packtpub.com/application-development/hands-test-driven-development-python-video?utm_source=github&utm_medium=repository&utm_campaign=9781789138313) 
 
 [まだmockで消耗してるの？mockを理解するための3つのポイント](https://note.crohaco.net/2015/python-mock/)

In [1]:
from unittest.mock import patch, Mock

### mock patch examples

In [2]:
class MyClass():
    def my_method(self):
        return False
 
class SomeOtherClassThatUsesMyClass():
    def method_under_test(self):
        myclass = MyClass()
        return myclass.my_method()

In [3]:
@patch.object(MyClass, 'my_method')
def test_my_method_shouldReturnTrue_whenMyMethodReturnsSomeValue(self, mock_my_method):
    mock_my_method.return_value=True
    some_other_class =  SomeOtherClassThatUsesMyClass()
    result = some_other_class.method_under_test()
    print(result)
    self.assertTrue(result)

In [4]:
my_test_class = SomeOtherClassThatUsesMyClass()
result = my_test_class.method_under_test()
print(result)

False


In [5]:
with patch.object(MyClass, 'my_method') as mock_my_method:
    mock_my_method.return_value=True
    my_test_class = SomeOtherClassThatUsesMyClass()
    result = my_test_class.method_under_test()
    print(result)

True


In [6]:
sample_mock = Mock(a=1, b=2)

In [7]:
sample_mock.a, sample_mock.b

(1, 2)

In [8]:
sample_mock = Mock(return_value=5)

In [9]:
sample_mock()

5

In [10]:
sample_mock = Mock(side_effect=range(2))

In [11]:
sample_mock()

0

In [12]:
sample_mock = Mock(return_value=ValueError)

In [13]:
sample_mock()

ValueError

### patch dict example

In [14]:
d = {'a': 1} #note the id!
print(d, id(d))

with patch.dict(d, {'b': 2, 'c': 3}):
    print(d, id(d))

{'a': 1} 4393935784
{'a': 1, 'b': 2, 'c': 3} 4393935784


### patch object example 
it can swap attributes within local variables, which is not possible with patch 

In [15]:
def test():
    test1 = Test()
    test2 = Test()
    with patch.object(test1, 'a', 2):
        print(test1.a)
        print(test2.a)
    with patch.object(test1, 'class_method', return_value=False):
        print(test1.class_method())

### mock - various ways of use

In [16]:
m = Mock()
m(1)

<Mock name='mock()' id='4393972344'>

In [17]:
m.assert_called_with(2)

AssertionError: Expected call: mock(2)
Actual call: mock(1)

### mock example using requests

In [ ]:
!pip install requests

You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import requests
def get_user_name(user, session):
    url = f"https://api.github.com/users/{user}"
    response = session.get(url)
    json_response = response.json()
    return json_response["name"]

In [ ]:
session = requests.Session()

In [ ]:
get_user_name("masayas", session)

In [ ]:
from unittest.mock import Mock

In [ ]:
spy = Mock(wraps=session)

In [ ]:
get_user_name("masayas", spy)

In [ ]:
spy.get.called

In [ ]:
spy.get.call_args

In [ ]:
spy.get.assert_called_with()

In [ ]:
from unittest.mock import MagicMock
from requests import Session
fake_session = MagicMock(spec=Session)
response_payload = {"name", "Fake Name"}

In [ ]:
get_user_name("masayas", fake_session)

In [ ]:
m = Mock(spec=Session)

### use of seal and how to patch

In [ ]:
import requests
def get_user_name2(user):
    url = f"https://api.github.com/users/{user}"
    session = requests.Session()
    response = session.get(url)
    json_response = response.json()
    return json_response["name"]

In [ ]:
get_user_name2("masayas")

In [ ]:
# Here, requests.Session instance has been replaced with patch (i.e., MagicMock), so mock.return_value.hogehoge will replace  the return value
from unittest.mock import patch, seal
response_payload = {"name": "Fake Name"}
with patch("requests.Session") as mock:
    mock.return_value.get.return_value.json.return_value = response_payload
    seal(mock)
    print(get_user_name2("masayas"))

# Understanding Patch

In [ ]:
from requests import Session

print(requests.Session)

with patch("requests.Session") as mock:
    print("patching started")
    print(Session)  # since this is not patched, we would not see it as MagicMock
    print(mock)
    print("patching finished")
    
print(requests.Session)
   

### import os example

In [ ]:
import os
def get_user():
    print(os.getenv("USER"))


In [ ]:
get_user()

In [ ]:
with patch.dict("os.environ", {"USER": "root"}):
    get_user()

# use of decorator and order of the arguments  

In [ ]:
import requests
def get_user_name2(user):
    url = f"https://api.github.com/users/{user}"
    session = requests.Session()
    response = session.get(url)
    json_response = response.json()
    return json_response["name"]


# Note the order of arguments inside test()
@patch("os.system")
@patch("requests.Session")
def test(mock_session, mock_system):
    payload = {"name": "Secret User"}
    mock_session.return_value.get.return_value.json.return_value = payload
    print(get_user_name2("masayas"))

In [ ]:
test()

In [ ]:
get_user_name2("masayas")